In [16]:
import sagemaker

#used to read s3 buckets if they are public from our local machine
# Use the AWS SDK for Python (Boto3) to create, configure, and manage AWS services,
# such as Amazon Elastic Compute Cloud (Amazon EC2) and Amazon Simple Storage Service (Amazon S3).
# The SDK provides an object-oriented API as well as low-level access to AWS services.

import boto3
# To utilize inbuilt algorithms
from sagemaker.amazon.amazon_estimator import get_image_uri  

#need to create sessions
from sagemaker.session import s3_input, Session

## Create a S3 bucket to store the dataset

In [17]:
bucket_name = 'benuramlbucket1'

In [18]:
# before create an s3 bucket first need to create a session
my_region = boto3.session.Session().region_name

In [19]:
# check  the region
print(my_region)

us-east-1


Get access of the S3 bucket

In [20]:
s3 = boto3.resource('s3')

In [21]:
# if only the region is us-east-1, s3 bucket should create on s3 console
try:
    if my_region == 'us-east-1':
        s3.create_bucket(Bucket=bucket_name)
    print('s3 bucket created successfully')
except Exception as e:
    print('s3 errror' , e) 

s3 bucket created successfully


ML Model output path

In [22]:
prefix = "mlmodel_first_aws"
output_path ="s3://{}/{}/output".format(bucket_name, prefix)
print(output_path)

s3://benuramlbucket1/mlmodel_first_aws/output


## Download Dataset

In [32]:
import pandas as pd
import urllib

try:
    urllib.request.urlretrieve('https://raw.githubusercontent.com/Benura2020/AWS-Playground/main/1.BankML/bank_clean.27f01fbbdf43271788427f3682996ae29ceca05d.csv', 'bank_clean.csv')
    print('Success: Downloaded bank_clean.csv')
except Exception as e:
    print('Data load error: ', e)
    
# this dataset download into sagemaker notebook storage (not to S3)

Success: Downloaded bank_clean.csv


In [33]:
try:
    model_data = pd.read_csv('./bank_clean.csv', index_col=0)
    print('Success: Data loaded intothe DataFrame')
except Exception as e:
    print('Data load error: ', e)

Success: Data loaded intothe DataFrame


In [34]:
model_data.head()

,age,campaign,pdays,previous,no_previous_contact,not_working,job_admin.,job_blue-collar,job_entrepreneur,job_housemaid,...,day_of_week_fri,day_of_week_mon,day_of_week_thu,day_of_week_tue,day_of_week_wed,poutcome_failure,poutcome_nonexistent,poutcome_success,y_no,y_yes
0,56,1,999,0,1,0,0,0,0,1,...,0,1,0,0,0,0,1,0,1,0
1,57,1,999,0,1,0,0,0,0,0,...,0,1,0,0,0,0,1,0,1,0
2,37,1,999,0,1,0,0,0,0,0,...,0,1,0,0,0,0,1,0,1,0
3,40,1,999,0,1,0,1,0,0,0,...,0,1,0,0,0,0,1,0,1,0
4,56,1,999,0,1,0,0,0,0,0,...,0,1,0,0,0,0,1,0,1,0


Dataset - a bank dataset. after promoting the age group, if they buy or not

In [36]:
#train and test split

import numpy as np
train_data, test_data = np.split(model_data.sample(frac=1, random_state=1729), [int(0.7 * len(model_data))])
print(train_data.shape, test_data.shape)

(28831, 61) (12357, 61)


/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/numpy/core/fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [39]:
# when working with aws sagemaker the dependent feature should be first column

## Saving Train And Test Into Buckets

In [38]:
# Train Data
import os

pd.concat([train_data['y_yes'], train_data.drop(['y_no', 'y_yes'],
                                                axis=1)],
                                                axis=1).to_csv('train.csv', index=False, header=False)
boto3.Session().resource('s3').Bucket(bucket_name).Object(os.path.join(prefix, 'train/train.csv')).upload_file('train.csv')
s3_input_train = sagemaker.TrainingInput(s3_data='s3://{}/{}/train'.format(bucket_name, prefix), content_type='csv')

In [41]:
# Test Data

pd.concat([test_data['y_yes'], test_data.drop(['y_no', 'y_yes'], axis=1) ], axis=1).to_csv('test.csv', index=False, header=False)
boto3.Session( ) . resource('s3' ) . Bucket(bucket_name).Object(os.path. join(prefix, 'test/test.csv') ) . upload_file('test.csv')
s3_input_test = sagemaker.TrainingInput(s3_data='s3://{}/{}/test'.format(bucket_name, prefix), content_type='csv') 

## Get the container of get_image_uri

In [42]:
# Building models with xgboost algorithm

# this line automatically looks for the XGBoost image URI and builds an XGBoost container.
# specify the repo_version depending on your preference.

container = get_image_uri(boto3.Session().region_name,
                            'xgboost',
                            repo_version='1.0-1')

The method get_image_uri has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


## Initialize Hyperparameteres

In [43]:
hyperparameters = {
	"max_depth":"5",
	"eta":"0.2",
	"gamma":"4",
	"min_child_weight":"6",
	"subsample":"0.7",
	"objective":"binary:logistic",
	"num_round":50
}

In [46]:
#construct a SageMaker estimator that calls the xgboost-container
estimator = sagemaker.estimator.Estimator(image_uri=container,
                                        hyperparameters=hyperparameters,
                                        role=sagemaker.get_execution_role(),
                                        train_instance_count=1,
                                        train_instance_type='ml.m5.2xlarge',
                                        train_volume_size=5, # 5 GB
                                        output_path=output_path,
                                        train_use_spot_instances=True,
                                        train_max_run=300,
                                        train_max_wait=600)

train_instance_count has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_instance_type has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_max_run has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_use_spot_instances has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_max_wait has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_volume_size has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


In [47]:
estimator.fit({'train': s3_input_train, 'validation': s3_input_test})

INFO:sagemaker:Creating training-job with name: sagemaker-xgboost-2024-03-29-03-30-13-106


2024-03-29 03:30:13 Starting - Starting the training job...
2024-03-29 03:30:28 Starting - Preparing the instances for training...
2024-03-29 03:31:03 Downloading - Downloading input data...
2024-03-29 03:31:18 Downloading - Downloading the training image...
2024-03-29 03:32:09 Training - Training image download completed. Training in progress.
2024-03-29 03:32:09 Uploading - Uploading generated training model.[2024-03-29 03:32:05.176 ip-10-0-152-96.ec2.internal:7 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None
INFO:sagemaker-containers:Imported framework sagemaker_xgboost_container.training
INFO:sagemaker-containers:Failed to parse hyperparameter objective value binary:logistic to Json.
Returning the value itself
INFO:sagemaker-containers:No GPUs detected (normal if no gpus installed)
INFO:sagemaker_xgboost_container.training:Running XGBoost Sagemaker in algorithm mode
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Determined delimiter of CSV input is ','
INFO:root

## Deploy model

In [48]:
# endpoint
my_predictor = estimator.deploy(initial_instance_count=1, instance_type='ml.m4.xlarge')

INFO:sagemaker:Creating model with name: sagemaker-xgboost-2024-03-29-03-37-09-400
INFO:sagemaker:Creating endpoint-config with name sagemaker-xgboost-2024-03-29-03-37-09-400
INFO:sagemaker:Creating endpoint with name sagemaker-xgboost-2024-03-29-03-37-09-400


------!

In [56]:
from sagemaker.serializers import CSVSerializer #input in the form of tabloid it needs to be converted into serial
test_data_array = test_data.drop(['y_no', 'y_yes'], axis=1).values #load the data into an array
# my_predictor.content_type = 'text/csv' # set the data type for an inference
my_predictor.serializer = CSVSerializer() # set the serializer type
predictions = my_predictor.predict(test_data_array).decode('utf-8') # predict!
predictions_array = np.fromstring(predictions[1:], sep=',') # and turn the prediction into an array
print(predictions_array.shape)

(12357,)


In [57]:
predictions_array

array([0.05214286, 0.05660191, 0.05096195, ..., 0.03436061, 0.02942475,
       0.03715819])

## Confusion Matrix

In [68]:
#from documentation
cm = pd.crosstab(index=test_data['y_yes'], columns=np.round(predictions_array), rownames=['Observed' ], colnames=['Predicted'])
tn = cm.iloc[0,0]; fn = cm.iloc[1,0]; tp = cm.iloc[1,1]; fp = cm.iloc[0, 1]; p = (tp+tn)/(tp+tn+fp+fn)*100
print("\n{0:<20}{1:<4.1f}%\n".format("Overall Classification Rate:", p))
print("{0:<15}{1:<15}{2:>8}".format("Predicted", "No Purchase", "Purchase"))
print("Observed")
print("{0:<15}{1:.0f}% ({2}){3:>6.0f}% ({4})".format("No Purchase", tn / (tn + fn) * 100, tn, fp / (tp + fp) * 100, fp))
print("{0:<16}{1:.0f}% ({2}){3:>7.0f}% ({4}) \n".format("Purchase", fn / (tn + fn) * 100, fn, tp / (tp + fp) * 100, tp))


Overall Classification Rate:89.7%

Predicted      No Purchase    Purchase
Observed
No Purchase    91% (10785)    34% (151)
Purchase        9% (1124)     66% (297) 



In [69]:
# delete endpoint
sagemaker.Session().delete_endpoint(my_predictor.endpoint)
bucket_to_delete = boto3.resource('s3').Bucket(bucket_name)
bucket_to_delete.objects.all().delete()

See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
INFO:sagemaker:Deleting endpoint with name: sagemaker-xgboost-2024-03-29-03-37-09-400


[{'ResponseMetadata': {'RequestId': 'MB3SJ4609BGMDWFN',
   'HostId': 'YP29GH6/jlx+cpsIDhylVuT78+f6M/ppaY0Wht4DKtzHxYR83yU+LVdYcG7L5dQgrTwMMCKcoyc=',
   'HTTPStatusCode': 200,
   'HTTPHeaders': {'x-amz-id-2': 'YP29GH6/jlx+cpsIDhylVuT78+f6M/ppaY0Wht4DKtzHxYR83yU+LVdYcG7L5dQgrTwMMCKcoyc=',
    'x-amz-request-id': 'MB3SJ4609BGMDWFN',
    'date': 'Fri, 29 Mar 2024 04:08:51 GMT',
    'content-type': 'application/xml',
    'transfer-encoding': 'chunked',
    'server': 'AmazonS3',
    'connection': 'close'},
   'RetryAttempts': 0},
  'Deleted': [{'Key': 'mlmodel_first_aws/output/sagemaker-xgboost-2024-03-29-03-30-13-106/debug-output/index/000000000/000000000020_worker_0.json'},
   {'Key': 'mlmodel_first_aws/output/sagemaker-xgboost-2024-03-29-03-30-13-106/debug-output/events/000000000000/000000000000_worker_0.tfevents'},
   {'Key': 'mlmodel_first_aws/output/sagemaker-xgboost-2024-03-29-03-30-13-106/output/model.tar.gz'},
   {'Key': 'mlmodel_first_aws/output/sagemaker-xgboost-2024-03-29-03-30-1